# Optimización por Enjambre por Partículas


In [27]:
import random

class Particle:
    """ 
    Clase que representa una partícula en el espacio de búsqueda.

    Atributos:
        position (list): Posición de la partícula en el espacio de búsqueda.
        velocity (list): Velocidad de la partícula en el espacio de búsqueda.
        best_position (list): Mejor posición de la partícula en el espacio de búsqueda.
        best_cost (float): Mejor costo de la partícula en el espacio de búsqueda.

    """
    def __init__(self, num_jobs, num_tasks):
        self.position = [[random.randint(0, num_tasks-1) for _ in range(num_tasks)] for _ in range(num_jobs)]
        self.velocity = self.initialize_velocity(num_jobs, num_tasks)
        self.best_position = self.position
        self.best_cost = float('inf')

    def initialize_velocity(self, num_jobs, num_tasks):
        """
        Inicializamos la velocidad de la partícula de forma aleatoria.

        :param num_jobs: Número de trabajos.
        :param num_tasks: Número de tareas.

        :return: Velocidad inicial de la partícula.

        """
        return [[random.uniform(-1, 1) for _ in range(num_tasks)] for _ in range(num_jobs)]


def calculate_total_time(particle, job_times):
    """
    Calculamos el tiempo total de ejecución de una determinada asignación.

    :param particle: Asignaciones de tareas a máquinas.
    :param job_times: Tiempos de ejecución de cada tarea.

    :return: Mayor tiempo total de ejecución de esa asignación.

    """
    max_time_per_machine = [0] * len(particle[0]) # Inicializamos a 0 el tiempo total de ejecución de cada máquina

    # Calculamos el tiempo total de ejecución de cada máquina
    for job, machine_particle in enumerate(particle):
        for task, machine in enumerate(machine_particle):
            max_time_per_machine[machine] += job_times[job][task]

    # Devolvemos el mayor tiempo total de ejecución de todas las máquinas
    return max(max_time_per_machine)

def update_velocity(particle, inertia_weight, cognitive_coefficient, social_coefficient, global_best_position):
    """
    Actualizamos la velocidad de la partícula.
    
    :param particle: Partícula a actualizar.
    :param inertia_weight: Coeficiente de la componente de inercia.
    :param cognitive_coefficient: Coeficiente de la componente cognitiva.
    :param social_coefficient: Coeficiente de la componente social.
    :param global_best_position: Mejor posición global.

    """

    for i in range(len(particle.velocity)):
        for j in range(len(particle.velocity[0])):

            # Generamos dos números aleatorios entre 0 y 1 uniformemente distribuidos
            r1, r2 = random.uniform(0, 1), random.uniform(0, 1)

            # Componente cognitiva
            cognitive_component = cognitive_coefficient * r1 * (particle.best_position[i][j] - particle.position[i][j])

            # Componente social
            social_component = social_coefficient * r2 * (global_best_position[i][j] - particle.position[i][j])

            # Componente de inercia
            inertia_component = inertia_weight * particle.velocity[i][j]

            # Actualizamos la velocidad de la partícula
            particle.velocity[i][j] = inertia_component  + cognitive_component + social_component

def update_position(particle):
    """
    Actualizamos la posición de la partícula.
    
    :param particle: Partícula a actualizar.
    """
    for i in range(len(particle.position)):
        for j in range(len(particle.position[0])):

            particle.position[i][j] += round(particle.velocity[i][j])

            # Nos aseguramos de que la posición de la partícula está dentro del espacio de búsqueda
            if particle.position[i][j] < 0:
                particle.position[i][j] = 0
            elif particle.position[i][j] >= len(particle.position[0]):
                particle.position[i][j] = len(particle.position[0]) - 1

def particle_swarm_optimization(job_times, num_particles=5, max_iterations=10, inertia_weight=0.7,
                                cognitive_coefficient=1.7, social_coefficient=0.7, seed=3, verbose=False):
    
    """
    Algoritmo de optimización por enjambre de partículas.
    
    :param job_times: Tiempos de ejecución de cada tarea.
    :param num_particles: Número de partículas.
    :param max_iterations: Número máximo de iteraciones.
    :param inertia_weight: Coeficiente de la componente de inercia.
    :param cognitive_coefficient: Coeficiente de la componente cognitiva.
    :param social_coefficient: Coeficiente de la componente social.
    :param seed: Semilla para la generación de números aleatorios.
    :param verbose: Si queremos mostrar información por pantalla.

    :return: Mejor asignación de tareas a máquinas y su tiempo de ejecución.
    """

    random.seed(seed)
    num_jobs = len(job_times)
    num_tasks = len(job_times[0])

    # Inicializamos las partículas
    particles = [Particle(num_jobs, num_tasks) for _ in range(num_particles)]

    # Inicializamos la mejor posición global y su coste
    global_best_position = None
    global_best_cost = float('inf')


    for iteracion in range(max_iterations):

        if verbose:
            print(f"Iteración {iteracion}/{max_iterations}\n")

        for particle in particles:

            # Calculamos el coste de cada partícula
            current_cost = calculate_total_time(particle.position, job_times)

            # Actualizamos la mejor posición de la partícula
            if current_cost < particle.best_cost:
                particle.best_position = particle.position
                particle.best_cost = current_cost

            # Actualizamos la mejor posición global
            if current_cost < global_best_cost:
                global_best_position = particle.position
                global_best_cost = current_cost

        for particle in particles:

            # Actualizamos la velocidad y la posición de cada partícula
            update_velocity(particle, inertia_weight, cognitive_coefficient, social_coefficient, global_best_position)
            update_position(particle)

            if verbose:
                print(f"Mejor posición partícula: {particle.best_position}, Tiempo: {particle.best_cost}")

        if verbose:
            print(f"\nMejor posición global: {global_best_position}, Tiempo: {global_best_cost}\n")
        

    return global_best_position, global_best_cost

### Ejemplo de uso

In [28]:
job_times_example = [[3, 1, 4], [7, 1, 8], [1, 2, 5]]

best_particle, best_cost = particle_swarm_optimization(job_times_example, verbose=True)
print("Mejor partícula:", best_particle)
print("Tiempo:", best_cost)

Iteración 0/10

Mejor posición partícula: [[0, 1, 2], [0, 1, 2], [1, 2, 2]], Tiempo: 20
Mejor posición partícula: [[1, 0, 1], [2, 0, 1], [1, 2, 1]], Tiempo: 20
Mejor posición partícula: [[0, 1, 1], [2, 2, 2], [0, 0, 2]], Tiempo: 20
Mejor posición partícula: [[1, 2, 2], [1, 2, 1], [1, 2, 1]], Tiempo: 13
Mejor posición partícula: [[1, 1, 2], [1, 1, 0], [0, 2, 1]], Tiempo: 12

Mejor posición global: [[1, 1, 2], [1, 1, 0], [0, 2, 1]], Tiempo: 12

Iteración 1/10

Mejor posición partícula: [[0, 1, 2], [0, 1, 2], [0, 2, 2]], Tiempo: 19
Mejor posición partícula: [[0, 1, 2], [2, 1, 0], [1, 2, 2]], Tiempo: 20
Mejor posición partícula: [[0, 1, 2], [2, 1, 1], [0, 1, 2]], Tiempo: 20
Mejor posición partícula: [[1, 2, 2], [1, 2, 1], [1, 2, 1]], Tiempo: 13
Mejor posición partícula: [[1, 1, 2], [1, 1, 0], [0, 2, 1]], Tiempo: 12

Mejor posición global: [[1, 1, 2], [1, 1, 0], [0, 2, 1]], Tiempo: 12

Iteración 2/10

Mejor posición partícula: [[0, 1, 2], [1, 1, 1], [0, 2, 2]], Tiempo: 19
Mejor posición par